<h1>Proyecto sobre interpretabilidad sobre modelos de caja negra</h1>

<p>En este proyecto nos dedicaremos a realizar el entrenamiento para los modelos de interpretabilidad de modelos de caja negra,en nuestro proyecto hemos decidido hacer el entrenamiento de modelos para tareas de clasifiacacion. Adelante en esta practica primero realizaremos el entrenamiento de los diferentes modelos de caja negra primero realizaremos el entrenamiento a un modelo de redes neuronales.</p>
<u><b>A continuacion el siguiente bloque de codigo es necesario ejecutarlo para el correcto funcionamiento de este notebook.</b></u>

In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in divide")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow import get_logger
get_logger().setLevel('ERROR')
import random
from tensorflow import random as tensorflow_random

random.seed(2398572)
tensorflow_random.set_seed(394867)

import pandas
from sklearn import model_selection

np.random.seed(43958734)
np.set_printoptions(threshold=10)
from tensorflow import keras as ke

from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist
import tensorflow as tf
from scipy.stats import spearmanr

<p>Ahora vamos a seleccionar el primer csv que contiene los datos y seleccionaremos los atributos que tendremos en cuenta para hacer el entrenamiento del algoritmo</p>

In [3]:
shill_bidding = pandas.read_csv("Shill Bidding Dataset.csv")
shill_bidding.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


Seleccionamos los atributos a tener encuenta a la hora de entrenar

In [4]:
atributos = shill_bidding.loc[:,'Bidder_Tendency':'Auction_Duration']
atributos = atributos.to_numpy()
print(atributos)


[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


<p>Vamos a seleccionar el atributo objetivo para el algoritmo</p>

In [5]:
objetivo = shill_bidding['Class']
objetivo = objetivo.to_numpy()
print(objetivo)

[0 0 0 ... 0 0 0]


Dividimos el modelo de entrenamiento con el modelo de pruebas

In [6]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.2)


Input indica el numero de dimensiones y dense indica cantidad de neuronas y la funcion de activacion, que por defecto es la identidad. Este ejemplo es para modelos de <b>clasificacion</b> 

La red neuronal posee una única capa (aparte de la de entrada) que proporciona como salida un array bidimensional None x 1 (es decir, para cada ejemplo proporciona un valor) y 9 parámetros (el peso de la conexión de cada entrada con la neurona de la capa más el sesgo de la neurona) entrenables (la red los aprenderá mediante el algoritmo de entrenamiento).

In [7]:
red_bidding = ke.Sequential()
red_bidding.add(ke.Input(shape=(9,)))
red_bidding.add(ke.layers.Dense(10, activation='relu'))
red_bidding.add(ke.layers.Dense(1, activation='sigmoid'))

Los pesos y sesgos de la red se guardan en el atributo weights (son los arrays asociados al argumento numpy en la estructura de datos guardada en ese atributo).

In [8]:
red_bidding.summary()
red_bidding.weights

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                100       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense/kernel:0' shape=(9, 10) dtype=float32, numpy=
 array([[ 0.16066974, -0.1832436 , -0.3538186 , ...,  0.340855  ,
          0.25578886,  0.5128347 ],
        [-0.1894975 , -0.0651882 , -0.12789819, ..., -0.19957277,
         -0.0312627 , -0.17042843],
        [-0.4715354 , -0.20407248,  0.29305267, ..., -0.50047326,
          0.43153203,  0.34140068],
        ...,
        [-0.01709646, -0.03771824, -0.15322047, ...,  0.49320394,
          0.24029279, -0.05415136],
        [ 0.2125234 ,  0.41607064, -0.06427649, ..., -0.03796834,
          0.12062675,  0.19724232],
        [ 0.27244765,  0.48618132,  0.47343248, ...,  0.20641726,
          0.0910573 ,  0.37640864]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 1) dtype=float32, numpy=
 array([[ 0.57531995],
        [ 0.17448515],
        [-0.48876932],
        [ 0.38587934],
  

Entrenamos la red neuronal con un optimizador y una funcion de perdida, en este caso al tratarse de un modelo de clasifiacion binaria seleccionamos la funcion de perdida como la entropia cruzada binaria

In [9]:
red_bidding.compile(optimizer='SGD', loss='binary_crossentropy',
                   metrics=['accuracy'])

Aqui entrenamos el modelo con los atributos de prueba seleccionados y el objetivo de entrenamiento

In [10]:
red_bidding.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=10)

Epoch 1/10
506/506 [==============================] - 1s 1ms/step - loss: 0.4077 - accuracy: 0.8728
Epoch 2/10
506/506 [==============================] - 1s 1ms/step - loss: 0.3151 - accuracy: 0.8934
Epoch 3/10
506/506 [==============================] - 1s 1ms/step - loss: 0.2429 - accuracy: 0.9120
Epoch 4/10
506/506 [==============================] - 1s 1ms/step - loss: 0.1768 - accuracy: 0.9456
Epoch 5/10
506/506 [==============================] - 1s 1ms/step - loss: 0.1307 - accuracy: 0.9608
Epoch 6/10
506/506 [==============================] - 1s 1ms/step - loss: 0.1028 - accuracy: 0.9636
Epoch 7/10
506/506 [==============================] - 1s 1ms/step - loss: 0.0861 - accuracy: 0.9654
Epoch 8/10
506/506 [==============================] - 1s 1ms/step - loss: 0.0766 - accuracy: 0.9650
Epoch 9/10
506/506 [==============================] - 1s 1ms/step - loss: 0.0697 - accuracy: 0.9689
Epoch 10/10
506/506 [==============================] - 1s 1ms/step - loss: 0.0660 - accuracy: 0.9711

Y ahora vamos a evaluar el modelo y ejecutar su prediccion con los atributos de prueba

In [11]:
red_bidding.evaluate(atributos_prueba, objetivo_prueba)
red_bidding.predict(atributos_prueba)
j = 0
vp=0
vn=0
fp=0
fn=0
for i in red_bidding.predict(atributos_prueba):
    
    if(i > 0.5):
        i = 1
    elif(i <= 0.5):
        i = 0
    if(i == objetivo_prueba[j] and i == 1):
        vp+=1
    elif(i !=objetivo_prueba[j] and i == 1):
        fp +=1
    elif(i ==objetivo_prueba[j] and i == 0):
        vn +=1
    elif(i !=objetivo_prueba[j] and i == 0):
        fn +=1
    j+=1
    
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))

40/40 [==============================] - 0s 1ms/step
Buenos resultados: 1232
Malos resultados: 33
La precision es:  0.9369369369369369


Aqui entrenaremos el 2º modelo de redes neuronales con el segundo csv que hemos escogido

In [12]:
wine_white = pandas.read_csv('winequality-white.csv', delimiter=";")
wine_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1001.0000,3.00,0.45,8.8,bueno
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,bueno
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,bueno
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,bueno
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,bueno


Como en el modelo anterior seleccionamos los atributos y el objetivo del csv y dividimos el conjunto de entrenamiento y de prueba, para este caso el modelo va a realizar una tarea de clasificacion multiclase

In [13]:
atributos1 = wine_white[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
atributos1 = atributos1.to_numpy()

objetivo = wine_white[['quality']]
objetivo = pandas.get_dummies(objetivo)
print(objetivo)
objetivo = objetivo.to_numpy().astype(float)

(atributos_entrenamiento, atributos_prueba1,
 objetivo_entrenamiento, objetivo_prueba1) = model_selection.train_test_split(
    atributos1, objetivo, test_size=.1)

      quality_bueno  quality_decente  quality_muy bueno  quality_perfecto
0                 1                0                  0                 0
1                 1                0                  0                 0
2                 1                0                  0                 0
3                 1                0                  0                 0
4                 1                0                  0                 0
...             ...              ...                ...               ...
4893              1                0                  0                 0
4894              0                1                  0                 0
4895              1                0                  0                 0
4896              0                0                  1                 0
4897              1                0                  0                 0

[4898 rows x 4 columns]


In [14]:
red_vino = ke.Sequential()
red_vino.add(ke.Input(shape=(11,)))
red_vino.add(ke.layers.Dense(10, activation='relu'))
# Aquie tantas neuronas como posibles clases
red_vino.add(ke.layers.Dense(4, activation='softmax'))

# Softmax es mejor activacion en clasificacion multiclase debido a que las
# probabilidades de todas las clases sumadas dan 1


In [15]:
red_vino.summary()
red_vino.weights

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                120       
                                                                 
 dense_3 (Dense)             (None, 4)                 44        
                                                                 
Total params: 164
Trainable params: 164
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense_2/kernel:0' shape=(11, 10) dtype=float32, numpy=
 array([[ 0.4539268 , -0.07710254,  0.15135956, ..., -0.16307437,
          0.01278824,  0.11068517],
        [ 0.05725986,  0.23043233, -0.40509826, ...,  0.21256512,
         -0.00953633, -0.43251088],
        [ 0.29988438, -0.3497378 ,  0.30682307, ..., -0.19849265,
         -0.49182558,  0.01923186],
        ...,
        [ 0.4283719 , -0.33114657, -0.39846885, ..., -0.27326483,
         -0.18029127,  0.5209586 ],
        [-0.38899636,  0.22119498,  0.16854131, ..., -0.24828452,
          0.11630654,  0.33638996],
        [-0.20374537,  0.20948946, -0.01595587, ..., -0.14216772,
         -0.21667987,  0.08175027]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_3/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-0.31399873,  0.11605477,  0.48944175, -0.63807887],
        [ 0.09330261, -0.25

Utilizamos diferentes funciones de perdida y optimizador debido a que realizamos una tarea diferente en este caso se trata de una clasificacion multiclase por lo que usaremos adam como optimizador y la entropia categorica para la funcion de perdida

In [16]:

def obtener_indice_maximo(arr):
    max_indice = np.argmax(arr)
    return max_indice

red_vino.compile( optimizer='adam', loss='categorical_crossentropy',
                   metrics=['accuracy'])

red_vino.fit(atributos_entrenamiento, objetivo_entrenamiento,
               batch_size=10, epochs=20)

red_vino.evaluate(atributos_prueba1, objetivo_prueba1)
predictions = red_vino.predict(atributos_prueba1)
# Esto lo realizamos para poder visualizar una aproximacion
for prediction, true_label in zip(predictions, objetivo_prueba1):
    print("Prediccion: ", obtener_indice_maximo(prediction), "Etiqueta real:", obtener_indice_maximo(true_label))
    # Realizar un algoritm que coja el indice del maximo de la lista y que devuelva un 1 en ese lugar

Epoch 1/20
441/441 [==============================] - 1s 1ms/step - loss: 18.6174 - accuracy: 0.4140
Epoch 2/20
441/441 [==============================] - 1s 1ms/step - loss: 5.9033 - accuracy: 0.4260
Epoch 3/20
441/441 [==============================] - 1s 1ms/step - loss: 6.5648 - accuracy: 0.4494
Epoch 4/20
441/441 [==============================] - 1s 1ms/step - loss: 4.6360 - accuracy: 0.4510
Epoch 5/20
441/441 [==============================] - 1s 1ms/step - loss: 4.5764 - accuracy: 0.4521
Epoch 6/20
441/441 [==============================] - 1s 1ms/step - loss: 4.9076 - accuracy: 0.4576
Epoch 7/20
441/441 [==============================] - 1s 1ms/step - loss: 3.6248 - accuracy: 0.4492
Epoch 8/20
441/441 [==============================] - 1s 1ms/step - loss: 4.5669 - accuracy: 0.4710
Epoch 9/20
441/441 [==============================] - 1s 1ms/step - loss: 4.4300 - accuracy: 0.4673
Epoch 10/20
441/441 [==============================] - 1s 1ms/step - loss: 5.1799 - accuracy: 0.476

# El siguiente modelo a realizar es random forest 

Ahora vamos a entrenar los modelos de clasificacion con RandomForest, este es un modelo de caja negra diferente a los dos anteriores y los entrenaremos con los mismo csv

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [18]:
# Cargar el archivo CSV
data = pandas.read_csv('Shill Bidding Dataset.csv')

# Separar los atributos (X) y el objetio (y)
X = data[['Bidder_Tendency','Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding','Auction_Bids', 'Starting_Price_Average', 'Early_Bidding','Winning_Ratio', 'Auction_Duration']]
X = X.to_numpy()
y = data['Class']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# n_stimators = numero de arboles y max_depth profundidad maxima de cada arbol 
randomForest_Bidding = RandomForestClassifier(n_estimators=100, max_depth=10)
print(X)

[[2.00000000e-01 4.00000000e-01 0.00000000e+00 ... 2.77778000e-05
  6.66666667e-01 5.00000000e+00]
 [2.43902440e-02 2.00000000e-01 0.00000000e+00 ... 1.31226852e-02
  9.44444444e-01 5.00000000e+00]
 [1.42857143e-01 2.00000000e-01 0.00000000e+00 ... 3.04166670e-03
  1.00000000e+00 5.00000000e+00]
 ...
 [5.55555560e-02 4.34782610e-02 0.00000000e+00 ... 1.56630291e-02
  0.00000000e+00 7.00000000e+00]
 [7.69230770e-02 8.69565220e-02 0.00000000e+00 ... 4.15013200e-04
  0.00000000e+00 7.00000000e+00]
 [1.63934430e-02 4.34782610e-02 0.00000000e+00 ... 3.40350529e-01
  0.00000000e+00 7.00000000e+00]]


In [19]:
# Entrenamos el modelo con los datos
randomForest_Bidding.fit(X_train, y_train )

RandomForestClassifier(max_depth=10)

In [20]:
predictions = randomForest_Bidding.predict(X_test)
# Esto lo realizamos para poder visualizar una aproximacion

vp=0
vn=0
fp=0
fn=0
for prediction, true_label in zip(predictions, y_test):
    if(prediction == true_label and prediction == 1  ):
        vp+=1
    elif(prediction != true_label and prediction == 1):
        fp +=1
    elif(prediction == true_label and prediction == 0):
        vn +=1
    elif(prediction != true_label and prediction == 0):
        fn +=1

        
print("Buenos resultados:", vp+vn)
print("Malos resultados:", fp+fn)
print("La precision es: ", vp/(vp+fp))


Buenos resultados: 1256
Malos resultados: 9
La precision es:  0.9424460431654677


In [21]:
# Cargar el archivo CSV
data = pandas.read_csv('winequality-white1.csv', delimiter=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [22]:

# Separar los atributos (X) y el objetio (y)
X = data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
X = X.to_numpy()
y = data['quality']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# n_stimators = numero de arboles y max_depth profundidad maxima de cada arbol 
randomForest_vinos = RandomForestClassifier(n_estimators=100, max_depth=10)
print(X)
for i in X_train:
    print(i)

[[ 7.    0.27  0.36 ...  3.    0.45  8.8 ]
 [ 6.3   0.3   0.34 ...  3.3   0.49  9.5 ]
 [ 8.1   0.28  0.4  ...  3.26  0.44 10.1 ]
 ...
 [ 6.5   0.24  0.19 ...  2.99  0.46  9.4 ]
 [ 5.5   0.29  0.3  ...  3.34  0.38 12.8 ]
 [ 6.    0.21  0.38 ...  3.26  0.32 11.8 ]]
[ 7.3   0.17  0.36 ...  3.14  0.41 12.4 ]
[6.3  0.25 0.44 ... 3.18 0.52 9.5 ]
[ 5.6   0.32  0.33 ...  3.25  0.49 11.1 ]
[ 6.9   0.19  0.35 ...  3.21  0.54 10.8 ]
[ 7.7   0.3   0.26 ...  3.2   0.5  10.4 ]
[6.5  0.46 0.24 ... 3.08 0.56 9.8 ]
[ 7.2   0.22  0.28 ...  3.08  0.59 11.3 ]
[ 5.9   0.44  0.36 ...  3.22  0.48 10.8 ]
[7.4  0.19 0.3  ... 3.14 0.49 9.1 ]
[7.   0.23 0.26 ... 3.22 0.55 9.5 ]
[ 6.8   0.18  0.38 ...  3.32  0.59 11.2 ]
[ 7.2   0.17  0.37 ...  3.08  0.46 11.  ]
[7.1  0.43 0.61 ... 3.11 0.45 8.7 ]
[ 6.5   0.23  0.2  ...  3.18  0.48 10.  ]
[7.9  0.64 0.46 ... 2.87 0.74 9.1 ]
[6.8  0.19 0.71 ... 2.85 0.5  9.5 ]
[ 5.7   0.32  0.38 ...  3.42  0.42 11.8 ]
[ 6.4   0.24  0.49 ...  3.01  0.98 10.5 ]
[7.5  0.28 0.39 ... 3.

[ 5.    0.35  0.25 ...  3.39  0.4  11.3 ]
[ 7.4   0.29  0.25 ...  3.11  0.4  11.  ]
[ 6.4   0.28  0.19 ...  3.31  0.35 10.2 ]
[9.2  0.28 0.46 ... 3.14 0.58 9.5 ]
[6.1  0.4  0.31 ... 3.22 0.77 9.5 ]
[7.   0.15 0.38 ... 3.18 0.42 9.8 ]
[ 8.1   0.33  0.36 ...  3.19  0.54 10.6 ]
[ 7.2   0.26  0.3  ...  3.33  0.43 12.1 ]
[6.6  0.18 0.26 ... 3.   0.43 9.4 ]
[6.8  0.29 0.49 ... 3.08 0.49 9.  ]
[7.   0.3  0.51 ... 3.07 0.52 9.6 ]
[ 8.1   0.29  0.49 ...  3.14  0.41 10.8 ]
[ 6.3   0.37  0.28 ...  3.29  0.46 11.6 ]
[ 7.    0.34  0.26 ...  3.08  0.35 11.6 ]
[6.2  0.36 0.45 ... 3.31 0.56 9.8 ]
[6.8   0.705 0.25  ... 3.36  0.52  9.5  ]
[ 8.2   0.29  0.49 ...  3.24  0.36 10.9 ]
[6.8  0.24 0.37 ... 3.2  0.5  9.4 ]
[6.6  0.23 0.27 ... 3.27 0.76 9.5 ]
[7.4  0.25 0.36 ... 3.01 0.48 9.  ]
[6.1  0.34 0.21 ... 3.02 0.53 9.4 ]
[ 6.5   0.16  0.33 ...  3.18  0.44 11.2 ]
[ 7.7   0.44  0.24 ...  3.12  0.43 11.3 ]
[ 6.6   0.36  0.24 ...  3.23  0.39 11.  ]
[7.3  0.19 0.27 ... 2.94 0.41 8.8 ]
[ 6.1   0.3   0.32 ...

[ 5.7   0.21  0.24 ...  3.65  0.72 10.1 ]
[5.8  0.15 0.28 ... 3.24 0.51 9.3 ]
[ 7.1   0.42  0.2  ...  3.23  0.47 13.4 ]
[ 7.    0.35  0.24 ...  3.35  0.38 11.  ]
[ 5.1   0.39  0.21 ...  3.5   0.45 12.5 ]
[ 6.3   0.23  0.31 ...  3.3   0.4  12.9 ]
[ 6.2   0.23  0.35 ...  3.37  0.43 11.  ]
[ 6.3   0.34  0.29 ...  3.29  0.53 10.1 ]
[8.   0.25 0.13 ... 2.96 0.46 9.7 ]
[ 7.5   0.35  0.37 ...  3.14  0.42 12.7 ]
[ 6.7   0.17  0.37 ...  3.26  0.6  10.8 ]
[8.1  0.34 0.28 ... 3.14 0.55 9.8 ]
[6.1  0.28 0.25 ... 3.25 0.43 9.  ]
[6.4  0.17 0.34 ... 3.25 0.49 9.6 ]
[ 8.1   0.24  0.38 ...  3.37  0.74 10.8 ]
[6.4  0.28 0.27 ... 3.14 0.46 8.7 ]
[6.5  0.23 0.36 ... 3.26 0.41 8.8 ]
[ 6.5   0.28  0.38 ...  3.03  0.42 13.1 ]
[ 7.2   0.24  0.24 ...  3.25  0.53 11.2 ]
[ 6.    0.28  0.52 ...  3.28  0.51 11.8 ]
[ 7.1   0.26  0.32 ...  3.17  0.37 11.2 ]
[ 6.4   0.22  0.32 ...  3.4   0.39 10.2 ]
[7.   0.3  0.38 ... 3.18 0.61 9.3 ]
[7.6  0.31 0.23 ... 3.16 0.5  9.7 ]
[6.4  0.17 0.34 ... 3.06 0.43 9.1 ]
[ 6.4   0.

[6.5  0.32 0.12 ... 3.22 0.32 9.  ]
[ 6.1   0.27  0.32 ...  3.36  0.4  12.5 ]
[6.9  0.24 0.33 ... 3.28 0.77 9.6 ]
[ 6.7   0.15  0.38 ...  3.09  0.53 11.4 ]
[ 8.1   0.22  0.43 ...  3.22  0.45 11.  ]
[6.   0.45 0.42 ... 3.02 0.4  9.  ]
[6.5  0.34 0.36 ... 3.44 0.55 9.3 ]
[6.7  0.26 0.39 ... 3.32 0.58 9.6 ]
[ 6.2   0.22  0.28 ...  3.19  0.48 10.5 ]
[ 9.    0.3   0.49 ...  2.94  0.32 11.5 ]
[ 8.1   0.12  0.38 ...  2.8   0.55 12.  ]
[ 7.9   0.28  0.41 ...  3.45  0.49 10.7 ]
[ 5.8   0.22  0.25 ...  3.37  0.58 10.4 ]
[6.6  0.26 0.25 ... 3.33 0.43 9.8 ]
[ 6.5   0.33  0.28 ...  3.24  0.32 12.2 ]
[ 5.9   0.65  0.23 ...  3.46  0.48 12.8 ]
[ 6.7   0.2   0.37 ...  3.11  0.45 11.4 ]
[6.8  0.25 0.24 ... 3.37 0.67 9.5 ]
[ 6.7   0.44  0.22 ...  3.26  0.53 12.8 ]
[ 7.1   0.27  0.27 ...  3.04  0.52 11.5 ]
[5.8  0.28 0.18 ... 3.23 0.58 9.55]
[6.8  0.32 0.18 ... 3.14 0.41 8.9 ]
[7.1  0.17 0.38 ... 3.35 0.52 9.6 ]
[ 7.2   0.32  0.3  ...  2.99  0.44 11.4 ]
[6.1  0.36 0.33 ... 3.15 0.78 9.5 ]
[6.6  0.45 0.43 

In [23]:
# Entrenamos el modelo con los datos
randomForest_vinos.fit(X_train, y_train )

RandomForestClassifier(max_depth=10)

In [24]:
predictions = randomForest_vinos.predict(X_test)
# Esto lo realizamos para poder visualizar una aproximacion
for prediction, true_label in zip(predictions, y_test):
    print("Prediccion: ", prediction, "Etiqueta real:", true_label)


Prediccion:  6 Etiqueta real: 7
Prediccion:  7 Etiqueta real: 7
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 4
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 6
Prediccion:  6 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Prediccion:  7 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 7
Prediccion:  6 Etiqueta real: 6
Prediccion:  5 Etiqueta real: 6
Prediccion:  6 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Prediccion:  6 Etiqueta real: 6
Prediccion:  7 Etiqueta real: 7
Prediccion:  5 Etiqueta real: 5
Prediccion:  5 Etiqueta real: 5
Predicci

# Este es el modelo LIME de interpretabilidad de modelos de caja negra

Este es el modelo de Lime que funciona podemos modificarlo averiguando la distancia con otro calculo como el de arriba y haciendo llamdas recursivas a funciones auxiliares

In [25]:

def perturb(datos, cabecera):
    k = random.randint(1, len(datos))
    datos_perturbados = datos.copy()
    for j in range(k):
        atributo_a_perturbar = random.randint(0,len(datos)-1)
        rango_maximo = abs(max(a[atributo_a_perturbar] for a in cabecera))
        rango_minimo = abs(min(a[atributo_a_perturbar] for a in cabecera))
        datos_perturbados[atributo_a_perturbar] = random.uniform(rango_minimo, rango_maximo)
    return datos_perturbados




In [26]:
# Cabe destacar que para que el comportamiento del modelo no sea aleatorio conviene inicializar la semilla siempre 
# En el mismo lougar
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge

def explicacion_del_modelo(N,f,datos, cabecera):
    random.seed(2398572)
    tensorflow_random.set_seed(394867)
    X = [] # Muestras perturbadas
    R = [] # Representaciones
    W = [] # Distancias entre muestra x y sus perturbaciones

    for i in range(N): # Por cada atributo a perturbar
        datos_perturbados= perturb(datos, cabecera)
        w = abs(sum(datos[atributo] - datos_perturbados[atributo] for atributo in range(0, len(datos)))) #Calculamos los pesos
        r = [0 if datos_perturbados[atributo] == datos[atributo] else 1 for atributo in range(0,len(datos))] #Indicamos su representacion
        X.append(datos_perturbados)
        R.append(r)
        W.append(w)
    Y_perturbada = []
    # Ahora vamos a dividir el comportamiento del algoritmo, dependiendo de si se trata de un modelo de redes neuronales o un randomForest
    for i in range(len(X)):
        if(isinstance(f,ke.Sequential)):
            x = X[i]
            array = np.array(x)
            x = array.reshape(1,-1)
            y = f.predict(x, verbose=0)
            if(len(y[0])>1): # Si nos encontramos con clasificacion multiclase, para averiguar la clase mas probable
                y = obtener_indice_maximo(y[0])
               
        else:
            x = X[i]
            array = np.array(x)
            x = array.reshape(1,-1)
            y = f.predict(x)
          

        Y_perturbada = np.append(Y_perturbada, y) 

    Y_perturbada = np.squeeze(Y_perturbada)
    G = Ridge()
    G.fit(R, Y_perturbada, sample_weight=W)
    pesos = G.coef_ #Importancia de los atributos
    
    return pesos

In [27]:

# Ejemplo utilizando el modelo "red_bidding" definido en el código anterior
parametros = explicacion_del_modelo(10, red_bidding, atributos[5809], atributos)

print("Parametros de la 1ª muestra:")


print("Red apuestas: ")
print(parametros)
print("--------------------------------")

params = explicacion_del_modelo(10, randomForest_Bidding, atributos[5809], atributos)

print("RandomForest Apuestas: ")
print(params)




print("Parametros de la 2ª muestra:")

parametros = explicacion_del_modelo(11, red_vino, atributos1[4800], atributos1)
print("Red vinos: ")
print(parametros)
print("--------------------------------")
params = explicacion_del_modelo(11, randomForest_vinos, atributos1[4809], atributos1)

print("RandomForest Vinos: ")
print(params)

Parametros de la 1ª muestra:
Red apuestas: 
[-0.31670122  0.1892155   0.30675382 -0.01101057  0.00710058  0.
 -0.23229795 -0.00252551  0.11321717]
--------------------------------
RandomForest Apuestas: 
[-0.30079047  0.21681666  0.00059828  0.23400087 -0.06915096  0.
 -0.15932433  0.27643354  0.03617409]
Parametros de la 2ª muestra:
Red vinos: 
[ 0.49822158  0.01354768 -0.827161   ... -0.17121512  0.34993245
  0.83109699]
--------------------------------
RandomForest Vinos: 
[ 2.46823910e-27 -1.94413804e-27  3.56250595e-27 ... -3.20587034e-27
 -1.55692285e-27 -2.64584738e-27]


<p><strong>Alpha:</strong> Este parámetro controla la fuerza de la regularización en el modelo Ridge. Cuanto mayor sea el valor de alpha, mayor será la penalización aplicada a los coeficientes. Un valor de alpha igual a 1.0 indica una regularización moderada.</p>

<p><strong>Copy_X:</strong> Es un parámetro booleano que indica si los datos de entrada deben ser copiados antes de ajustar el modelo. Si se establece en True, se realiza una copia de los datos; si se establece en False, los datos se sobrescriben.</p>

<p><strong>Fit_intercept:</strong> Es un parámetro booleano que indica si se debe ajustar un intercepto en el modelo. Si se establece en True, el modelo incluirá un término constante; si se establece en False, el modelo no incluirá un término constante.</p>

<p><strong>Max_iter:</strong> Es el número máximo de iteraciones permitidas durante el ajuste del modelo. Si se establece en None, el modelo ajustará hasta que la convergencia sea alcanzada.</p>

<p><strong>Positive:</strong> Es un parámetro booleano que indica si los coeficientes deben ser no negativos. Si se establece en True, se restringe que los coeficientes sean siempre positivos.</p>

<p><strong>Random_state:</strong> Este parámetro controla la semilla utilizada por el generador de números aleatorios. Si se establece en None, el generador de números aleatorios utilizará una semilla diferente en cada ejecución.</p>

<p><strong>Solver:</strong> Especifica el algoritmo a utilizar en el ajuste del modelo. Si se establece en 'auto', el modelo seleccionará automáticamente el mejor algoritmo según los datos y el problema en cuestión.</p>

<p><strong>Tol:</strong> Especifica la tolerancia para el criterio de parada en el ajuste del modelo. Si la diferencia entre dos iteraciones consecutivas es menor que la tolerancia, el ajuste se considera convergido.</p>

<p><strong>Coef_:</strong> Especifica el peso que en esta caso se trata de la importancia de cada atributo a la hora de tomar decisiones para clasificar las muestras</p>


# Aqui tenemos los calculos requeridos sobre el trabajo

 <h1>Resumen de cálculos</h1>

  <i><b>1. Identidad:</b></i>
  <p>Verifica si las explicaciones son idénticas para objetos idénticos.</p>

 <i><b>2. Separabilidad:</b></i>
  <p>Verifica que las explicaciones sean diferentes para objetos no idénticos.</p>

 <i><b>3. Estabilidad:</b></i>
  <p>Evalúa si objetos similares tienen explicaciones similares mediante la correlación de Spearman.</p>

  <i><b>4. Selectividad:</b></i>
  <p>Mide el efecto de eliminar variables relevantes en la predicción mediante el cálculo del área bajo la curva (AUC).</p>

  <i><b>5. Coherencia:</b></i>
  <p>Calcula la coherencia de una muestra al eliminar características no importantes.</p>

  <i><b>6. Completitud (Coherencia / Explicación):</b></i>
  <p>Calcula el error de explicacion con respecto al error de prediccion</p>

 <i><b>7. Congruencia:</b></i>
  <p>Calcula la desviación estándar de la coherencia como un proxy de congruencia.</p>


In [28]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

def identidad_(muestraA, muestraB, explicacionA, explicacionB):
    res = False
    distancia = np.linalg.norm(muestraA-muestraB)

    if distancia == 0:
        res = np.linalg.norm(explicacionA - explicacionB) == 0
    return res

def separabilidad_(muestraA, muestraB, explicacionA, explicacionB):
    res = True
    distancia = np.linalg.norm(muestraA-muestraB)
   
    if distancia != 0:
        res != np.linalg.norm(explicacionA - explicacionB) == 0
    return res

def estabilidad_(distancia_muestra, distancia_explicaciones):


    # Calcula la correlación de Spearman
    rho, _ = spearmanr(distancia_muestra, distancia_explicaciones)

    # Verifica si la correlación es mayor que cero para cada elemento
    print(rho)
    if np.all(rho != 0):
        return True
    else:
        return False


def selectividad_(muestras, n_caracteristicas, modelo, modo):
    auc_scores = []  # Inicializar la lista para almacenar los AUC scores
    
    for i in range(n_caracteristicas):
        muestras_modificadas = np.copy(muestras)
        muestras_modificadas[:, i] = 0
        
         # Obtener las predicciones del modelo con la característica i eliminada
        y_pred = modelo.predict(muestras_modificadas)
        
        # Obtener las etiquetas reales correspondientes al modo
        if modo == "b":
            y_true = objetivo_apuestas
             # Calcular el área bajo la curva (AUC) y almacenarlo en auc_scores
            auc = roc_auc_score(y_true, y_pred)
            auc_scores.append(auc)
        else:
            aux = []
            aux_score = []
            for i, j in zip(objetivo_vinos,y_pred):
                aux.append(obtener_indice_maximo(i))
                aux_score.append(obtener_indice_maximo(j))
            y_true = aux
            y_pred = aux_score

             # Calcular la precision y almacenarlo en auc_scores
            accuracy = accuracy_score(y_true, y_pred)
            auc_scores.append(accuracy)
           
    
    if auc_scores:
        selectividad = np.mean(auc_scores)
    else:
        selectividad = 0.0
    
    return selectividad


def coherencia_(prediccion_original, prediccion_nueva, objetivo):
    # Obtener las predicciones del modelo para la señal original y la nueva señal
    # Calcular el error de predicción para la señal original
    
    error_prediccion_original = np.abs(prediccion_original - objetivo)
    
    error_prediccion_nueva = np.abs(prediccion_nueva - objetivo)
    
    coherencia = np.abs(error_prediccion_original - error_prediccion_nueva)
    return coherencia
def completitud_(prediccion_original, explicacion_error , objetivo):
    # Hallar el porcentaje de fallo de explicacion con respecto al error de prediccion
    error_prediccion = np.mean(np.abs(prediccion_original - objetivo))

    completitud = np.abs(explicacion_error / error_prediccion) 
    return completitud
    
def congruencia_(coherencia):
    #Congruencia
    alpha = np.mean(coherencia)  # alpha_values es una lista/array con los valores de coherencia alpha_i

    # Calcular la desviación estándar de la coherencia
    N = len(data)
    delta = np.sqrt(np.mean((coherencia - alpha)**2) / N)  # N es el número total de muestras
    return delta


In [29]:
# Escogemos 256 muestras de prueba para las metricas
muestras_apuestas = []
muestras_vinos = []
objetivo_apuestas = []
objetivo_vinos = []

for i in range(0, 256):
    muestras_apuestas.append(atributos_prueba[i])
    objetivo_apuestas.append(objetivo_prueba[i])
    muestras_vinos.append(X_test[i])
    objetivo_vinos.append(objetivo_prueba1[i])

muestras_apuestas = np.array(muestras_apuestas)
objetivo_apuestas = np.array(objetivo_apuestas)
muestras_vinos = np.array(muestras_vinos)
objetivo_vinos = np.array(objetivo_vinos)


In [30]:
# Identidad
ide = True 
res = ""
for i in range(0, 256):
    ide &= identidad_(muestras_apuestas[i], muestras_apuestas[i], explicacion_del_modelo(10 , red_bidding, muestras_apuestas[i], atributos_prueba),  explicacion_del_modelo(10 , red_bidding, muestras_apuestas[i], atributos_prueba))    
    ide &= identidad_(muestras_apuestas[i], muestras_apuestas[i], explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[i], atributos_prueba),  explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[i], atributos_prueba))    
    ide &= identidad_(muestras_vinos[i], muestras_vinos[i], explicacion_del_modelo(10 , red_vino, muestras_vinos[i], X_test),  explicacion_del_modelo(10 , red_vino, muestras_vinos[i], X_test))    
    ide &= identidad_(muestras_vinos[i], muestras_vinos[i], explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[i], X_test),  explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[i], X_test))    
if(ide == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de identidad para todos los modelos.")

Se cumple la metrica de identidad para todos los modelos.


In [31]:
# Separabilidad
sep = True
res = ""
for i in range(0, 255):
    sep &= separabilidad_(muestras_apuestas[i], muestras_apuestas[i+1], explicacion_del_modelo(10 , red_bidding, muestras_apuestas[i], atributos_prueba),  explicacion_del_modelo(10 , red_bidding, muestras_apuestas[i+1], atributos_prueba))
    sep &= separabilidad_(muestras_apuestas[i], muestras_apuestas[i+1], explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[i], atributos_prueba),  explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[i+1], atributos_prueba))    
    sep &= separabilidad_(muestras_vinos[i], muestras_vinos[i+1], explicacion_del_modelo(10 , red_vino, muestras_vinos[i], X_test),  explicacion_del_modelo(10 , red_vino, muestras_vinos[i+1], X_test))    
    sep &= separabilidad_(muestras_vinos[i], muestras_vinos[i+1], explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[i], X_test),  explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[i+1], X_test))    
    
if(sep == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de separabilidad para todos los modelos.")

Se cumple la metrica de separabilidad para todos los modelos.


In [32]:
# Estabilidad
est = True
dist_muestrasBidding = []
dist_muestrasVinos = []
dist_explicaciones_redBidding = []
dist_explicaciones_RFBidding = []
dist_explicaciones_redVinos = []
dist_explicaciones_RFVinos = []
for i in range(0, 25):
    for j in range(1, 256):
        dist_muestrasBidding_aux = np.abs(muestras_apuestas[i] - muestras_apuestas[j])
        dist_muestrasBidding.append(dist_muestrasBidding_aux)
        
        dist_explicaciones_redBidding_aux = np.abs(explicacion_del_modelo(10 , red_bidding, muestras_apuestas[i], atributos_prueba)-  explicacion_del_modelo(10 , red_bidding, muestras_apuestas[j], atributos_prueba))
        dist_explicaciones_redBidding.append(dist_explicaciones_redBidding_aux)
    est &= estabilidad_(dist_muestrasBidding, dist_explicaciones_redBidding)

    dist_muestrasBidding = []
    dist_muestrasVinos = []
    dist_explicaciones_redBidding = []
    dist_explicaciones_RFBidding = []
    dist_explicaciones_redVinos = []
    dist_explicaciones_RFVinos = []

    
if(est == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de estabilidad para todos los modelos.")

[[1.         0.25456385 0.26657754 ... 0.07692685 0.22672743 0.0689556 ]
 [0.25456385 1.         0.4875626  ... 0.14485702 0.72177582 0.22250763]
 [0.26657754 0.4875626  1.         ... 0.09671881 0.49879374 0.43815409]
 ...
 [0.07692685 0.14485702 0.09671881 ... 1.         0.14347354 0.19871324]
 [0.22672743 0.72177582 0.49879374 ... 0.14347354 1.         0.2213988 ]
 [0.0689556  0.22250763 0.43815409 ... 0.19871324 0.2213988  1.        ]]
[[ 1.          0.25524278  0.2737862  ...  0.18265323 -0.0486247
   0.11091612]
 [ 0.25524278  1.          0.40073006 ...  0.31806457  0.41822568
   0.20969917]
 [ 0.2737862   0.40073006  1.         ...  0.51896396  0.36316515
   0.45028248]
 ...
 [ 0.18265323  0.31806457  0.51896396 ...  1.          0.23493154
   0.15740288]
 [-0.0486247   0.41822568  0.36316515 ...  0.23493154  1.
   0.32518744]
 [ 0.11091612  0.20969917  0.45028248 ...  0.15740288  0.32518744
   1.        ]]
[[ 1.         -0.01830256 -0.08130283 ... -0.03050521  0.00253779
  -0.00

[[1.         0.24954941 0.27445738 ... 0.11896914 0.08360945 0.11143873]
 [0.24954941 1.         0.50893634 ... 0.28033534 0.66880701 0.18804647]
 [0.27445738 0.50893634 1.         ... 0.25811699 0.45711522 0.44764333]
 ...
 [0.11896914 0.28033534 0.25811699 ... 1.         0.26741692 0.30775098]
 [0.08360945 0.66880701 0.45711522 ... 0.26741692 1.         0.17699601]
 [0.11143873 0.18804647 0.44764333 ... 0.30775098 0.17699601 1.        ]]
[[ 1.          0.24834904  0.27445738 ...  0.04583033  0.23095184
   0.1485179 ]
 [ 0.24834904  1.          0.51529414 ...  0.08883048  0.76362966
   0.12494083]
 [ 0.27445738  0.51529414  1.         ... -0.06416624  0.4993306
   0.42986871]
 ...
 [ 0.04583033  0.08883048 -0.06416624 ...  1.          0.07878141
   0.13953509]
 [ 0.23095184  0.76362966  0.4993306  ...  0.07878141  1.
   0.0682709 ]
 [ 0.1485179   0.12494083  0.42986871 ...  0.13953509  0.0682709
   1.        ]]
[[ 1.          0.2642853   0.27445738 ...  0.05501037  0.23231192
   0.114

In [33]:
for i in range(0, 25):
    for j in range(1, 256):
        dist_muestrasBidding_aux = np.abs(muestras_apuestas[i] - muestras_apuestas[j])
        dist_muestrasBidding.append(dist_muestrasBidding_aux)
        dist_explicaciones_RFBidding_aux = np.abs(explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[i], atributos_prueba)-  explicacion_del_modelo(10 , randomForest_Bidding, muestras_apuestas[j], atributos_prueba))
        dist_explicaciones_RFBidding.append(dist_explicaciones_RFBidding_aux)
    est &= estabilidad_(dist_muestrasBidding, dist_explicaciones_RFBidding)
    dist_muestrasBidding = []
    dist_muestrasVinos = []
    dist_explicaciones_redBidding = []
    dist_explicaciones_RFBidding = []
    dist_explicaciones_redVinos = []
    dist_explicaciones_RFVinos = []
if(est == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de estabilidad para todos los modelos.")

[[1.         0.25456385 0.26657754 ... 0.2082064  0.18742865 0.143259  ]
 [0.25456385 1.         0.4875626  ... 0.57472846 0.7039625  0.31984167]
 [0.26657754 0.4875626  1.         ... 0.51619965 0.4751725  0.4164679 ]
 ...
 [0.2082064  0.57472846 0.51619965 ... 1.         0.53516679 0.21804542]
 [0.18742865 0.7039625  0.4751725  ... 0.53516679 1.         0.07123462]
 [0.143259   0.31984167 0.4164679  ... 0.21804542 0.07123462 1.        ]]
[[1.         0.25524278 0.2737862  ... 0.23162806 0.21532259 0.26074833]
 [0.25524278 1.         0.40073006 ... 0.50645643 0.5877566  0.39322116]
 [0.2737862  0.40073006 1.         ... 0.554885   0.48564116 0.43296577]
 ...
 [0.23162806 0.50645643 0.554885   ... 1.         0.64887339 0.15869566]
 [0.21532259 0.5877566  0.48564116 ... 0.64887339 1.         0.38133957]
 [0.26074833 0.39322116 0.43296577 ... 0.15869566 0.38133957 1.        ]]
[[ 1.         -0.01830256 -0.08130283 ... -0.07026877 -0.01182649
   0.01495566]
 [-0.01830256  1.         -0.46

[[1.         0.24834904 0.27445738 ... 0.21234328 0.17753439 0.2646907 ]
 [0.24834904 1.         0.51529414 ... 0.5873926  0.71502738 0.38501151]
 [0.27445738 0.51529414 1.         ... 0.47990295 0.46175113 0.42557187]
 ...
 [0.21234328 0.5873926  0.47990295 ... 1.         0.71877203 0.35250022]
 [0.17753439 0.71502738 0.46175113 ... 0.71877203 1.         0.50359831]
 [0.2646907  0.38501151 0.42557187 ... 0.35250022 0.50359831 1.        ]]
[[1.         0.2642853  0.27445738 ... 0.21804783 0.18202616 0.2650385 ]
 [0.2642853  1.         0.51180053 ... 0.57321305 0.71080949 0.34982324]
 [0.27445738 0.51180053 1.         ... 0.49046765 0.46443541 0.43144893]
 ...
 [0.21804783 0.57321305 0.49046765 ... 1.         0.67434819 0.30288806]
 [0.18202616 0.71080949 0.46443541 ... 0.67434819 1.         0.38904497]
 [0.2650385  0.34982324 0.43144893 ... 0.30288806 0.38904497 1.        ]]
[[ 1.00000000e+00 -4.27168654e-02 -9.57096783e-02 ... -7.05217318e-02
   5.17552175e-04  3.03173378e-02]
 [-4.27

In [34]:
for i in range(0, 25):
    for j in range(1, 256):
        dist_muestrasVinos_aux = np.abs(muestras_vinos[i] - muestras_vinos[j])
        dist_muestrasVinos.append(dist_muestrasVinos_aux)
        dist_explicaciones_redVinos_aux = np.abs(explicacion_del_modelo(10 , red_vino, muestras_vinos[i], X_test)-  explicacion_del_modelo(10 , red_vino, muestras_vinos[j], X_test))
        dist_explicaciones_redVinos.append(dist_explicaciones_redVinos_aux)

    est &= estabilidad_(dist_muestrasVinos, dist_explicaciones_redVinos)
    dist_muestrasBidding = []
    dist_muestrasVinos = []
    dist_explicaciones_redBidding = []
    dist_explicaciones_RFBidding = []
    dist_explicaciones_redVinos = []
    dist_explicaciones_RFVinos = []
if(est == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de estabilidad para todos los modelos.")

[[ 1.         -0.00487567 -0.06734976 ...  0.07088239  0.06226873
   0.06437908]
 [-0.00487567  1.          0.02032067 ...  0.04614434  0.01484545
   0.06285154]
 [-0.06734976  0.02032067  1.         ... -0.05547805  0.02774402
  -0.03383416]
 ...
 [ 0.07088239  0.04614434 -0.05547805 ...  1.          0.63974547
   0.78366921]
 [ 0.06226873  0.01484545  0.02774402 ...  0.63974547  1.
   0.32603259]
 [ 0.06437908  0.06285154 -0.03383416 ...  0.78366921  0.32603259
   1.        ]]
[[ 1.         -0.08318286  0.25147384 ...  0.07597735 -0.03091539
   0.07128792]
 [-0.08318286  1.          0.13373075 ... -0.07569195 -0.07216027
  -0.13551294]
 [ 0.25147384  0.13373075  1.         ... -0.09233445 -0.08646854
   0.00533461]
 ...
 [ 0.07597735 -0.07569195 -0.09233445 ...  1.          0.52956173
   0.55891344]
 [-0.03091539 -0.07216027 -0.08646854 ...  0.52956173  1.
   0.32636601]
 [ 0.07128792 -0.13551294  0.00533461 ...  0.55891344  0.32636601
   1.        ]]
[[ 1.          0.09486721  0.082

[[ 1.          0.08631494  0.10633834 ...  0.04558239  0.04312982
   0.00567205]
 [ 0.08631494  1.          0.26104993 ...  0.15594021  0.12219521
   0.12801823]
 [ 0.10633834  0.26104993  1.         ... -0.03448003 -0.03086759
   0.02727151]
 ...
 [ 0.04558239  0.15594021 -0.03448003 ...  1.          0.494167
   0.78023093]
 [ 0.04312982  0.12219521 -0.03086759 ...  0.494167    1.
   0.15350624]
 [ 0.00567205  0.12801823  0.02727151 ...  0.78023093  0.15350624
   1.        ]]
[[ 1.          0.05312521 -0.07031535 ...  0.10306874  0.04003207
   0.07515332]
 [ 0.05312521  1.          0.10490823 ...  0.11068842  0.12430558
   0.1002624 ]
 [-0.07031535  0.10490823  1.         ...  0.05905958  0.01324633
   0.0600391 ]
 ...
 [ 0.10306874  0.11068842  0.05905958 ...  1.          0.8426398
   0.86587277]
 [ 0.04003207  0.12430558  0.01324633 ...  0.8426398   1.
   0.62255478]
 [ 0.07515332  0.1002624   0.0600391  ...  0.86587277  0.62255478
   1.        ]]
[[ 1.          0.037451   -0.021077

In [35]:
for i in range(0, 25):
    for j in range(1, 256):
        dist_muestrasVinos_aux = np.abs(muestras_vinos[i] - muestras_vinos[j])
        dist_muestrasVinos.append(dist_muestrasVinos_aux)
        dist_explicaciones_RFVinos_aux = np.abs(explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[i], X_test)-  explicacion_del_modelo(10 , randomForest_vinos, muestras_vinos[j], X_test))
        dist_explicaciones_RFVinos.append(dist_explicaciones_RFVinos_aux)
    est &= estabilidad_(dist_muestrasVinos, dist_explicaciones_RFVinos)
    dist_muestrasBidding = []
    dist_muestrasVinos = []
    dist_explicaciones_redBidding = []
    dist_explicaciones_RFBidding = []
    dist_explicaciones_redVinos = []
    dist_explicaciones_RFVinos = []
if(est == True):
    res = "Se cumple"
else:
    res = "No se cumple"

print(res, "la metrica de estabilidad para todos los modelos.")

[[ 1.         -0.00487567 -0.06734976 ... -0.10924708 -0.15804291
  -0.05813278]
 [-0.00487567  1.          0.02032067 ...  0.00537842 -0.14885026
  -0.13948297]
 [-0.06734976  0.02032067  1.         ...  0.19068243  0.15888702
   0.21918969]
 ...
 [-0.10924708  0.00537842  0.19068243 ...  1.          0.29266911
   0.24341663]
 [-0.15804291 -0.14885026  0.15888702 ...  0.29266911  1.
   0.37094145]
 [-0.05813278 -0.13948297  0.21918969 ...  0.24341663  0.37094145
   1.        ]]
[[ 1.         -0.08318286  0.25147384 ... -0.0601561   0.10002835
   0.1962918 ]
 [-0.08318286  1.          0.13373075 ... -0.04128283  0.32822108
  -0.16949714]
 [ 0.25147384  0.13373075  1.         ... -0.09129398  0.06775242
   0.10507303]
 ...
 [-0.0601561  -0.04128283 -0.09129398 ...  1.          0.23874517
   0.32944166]
 [ 0.10002835  0.32822108  0.06775242 ...  0.23874517  1.
   0.28255751]
 [ 0.1962918  -0.16949714  0.10507303 ...  0.32944166  0.28255751
   1.        ]]
[[ 1.          0.09486721  0.082

[[ 1.          0.08631494  0.10633834 ... -0.0254446  -0.0624562
  -0.08344762]
 [ 0.08631494  1.          0.26104993 ...  0.08782732  0.41402004
   0.40037937]
 [ 0.10633834  0.26104993  1.         ...  0.01158463  0.04700748
   0.06203007]
 ...
 [-0.0254446   0.08782732  0.01158463 ...  1.          0.48270299
   0.31444125]
 [-0.0624562   0.41402004  0.04700748 ...  0.48270299  1.
   0.38936503]
 [-0.08344762  0.40037937  0.06203007 ...  0.31444125  0.38936503
   1.        ]]
[[ 1.          0.05312521 -0.07031535 ...  0.12866732 -0.04485542
  -0.15771802]
 [ 0.05312521  1.          0.10490823 ...  0.04084875  0.14574875
  -0.02365061]
 [-0.07031535  0.10490823  1.         ...  0.09851714 -0.03734709
   0.19461619]
 ...
 [ 0.12866732  0.04084875  0.09851714 ...  1.         -0.13589766
   0.07079279]
 [-0.04485542  0.14574875 -0.03734709 ... -0.13589766  1.
   0.17449083]
 [-0.15771802 -0.02365061  0.19461619 ...  0.07079279  0.17449083
   1.        ]]
[[ 1.          0.037451   -0.0210

Esto quiere decir que para cada cambio en la muestra, la explicacion varia de la misma manera.

In [36]:
# Selectividad (precision)

def softmax(x, axis=None):
    # Asegurarse de que x tenga al menos dos dimensiones
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

sel_redBidding = selectividad_(muestras_apuestas,9, red_bidding,"b")
sel_randomBidding = selectividad_(muestras_apuestas,9, red_bidding,"b")
sel_redVinos = selectividad_(muestras_vinos,10, red_vino,"m")
sel_randomVinos = selectividad_(muestras_vinos,10, randomForest_vinos,"m")

print("La selectividad para la red de apuestas es: " , sel_redBidding)

print("La selectividad para randomForest de apuestas es: " , sel_randomBidding)

print("La selectividad para la red de vinos es: " , sel_redVinos)

print("La selectividad para randomForest de vinos es: " , sel_randomVinos)

8/8 [==============================] - 0s 1ms/step
La selectividad para la red de apuestas es:  0.9805590364800891
La selectividad para randomForest de apuestas es:  0.9805590364800891
La selectividad para la red de vinos es:  0.3765625
La selectividad para randomForest de vinos es:  0.484375


In [37]:
# Red_bidding Coherencia, Completitud, Congruencia
print("Estas son las metricas de coherencia, completitud y congruencia de la red neuronal de apuestas")
co = 0.0
cohe = []
prediccion_original = red_bidding.predict(muestras_apuestas)
muestras_modificadas = np.copy(muestras_apuestas)
    # Eliminacio de caracteristicas no importantes para red de apuestas
muestras_modificadas[:, 0] = 0
muestras_modificadas[:, 6] = 0
muestras_modificadas[:, 7] = 0
prediccion_nueva = red_bidding.predict(muestras_modificadas)
print("La coherencia se enseña a continuacion por cada muestra: ")
for j in range(0, 256):
    
    co = coherencia_(prediccion_original[j], prediccion_nueva[j], objetivo_apuestas[j])
    cohe.append(co)
    print(j,":",co)
    

prediccion_original = red_bidding.predict(muestras_apuestas)
explicacion_error = []

for i in range(0, 256):
    exp = explicacion_del_modelo(10,red_bidding, muestras_apuestas[i], muestras_apuestas)
    exp2 = explicacion_del_modelo(10,red_bidding, muestras_modificadas[i], muestras_modificadas)
    explicacion_error.append(np.abs(exp - exp2))
    
explicacion_error = np.mean(explicacion_error)
comple = completitud_(prediccion_original,explicacion_error, objetivo_apuestas)
print("La completitud se muestra a continuacion:")
print(comple)
#Congruencia
congru = congruencia_(cohe)
print("La congruencia es: ", congru)

Estas son las metricas de coherencia, completitud y congruencia de la red neuronal de apuestas
8/8 [==============================] - 0s 1ms/step
La coherencia se enseña a continuacion por cada muestra: 
0 : [0.00107427]
1 : [0.0005334]
2 : [0.01256791]
3 : [0.192263]
4 : [0.01524867]
5 : [0.00088601]
6 : [0.00045936]
7 : [0.09827983]
8 : [0.21944124]
9 : [0.01167106]
10 : [0.00056252]
11 : [0.00428031]
12 : [0.00049821]
13 : [9.8671764e-05]
14 : [0.0003299]
15 : [0.00161545]
16 : [6.664451e-05]
17 : [0.00361826]
18 : [0.00131988]
19 : [0.00129872]
20 : [0.00948452]
21 : [0.00017708]
22 : [0.00328514]
23 : [0.00074382]
24 : [0.00110333]
25 : [0.00071274]
26 : [1.1726515e-05]
27 : [0.00106604]
28 : [0.07593107]
29 : [7.145456e-05]
30 : [0.00019561]
31 : [0.01590146]
32 : [0.00366184]
33 : [0.00153615]
34 : [0.00090538]
35 : [0.00717275]
36 : [0.00156649]
37 : [2.0700041e-05]
38 : [0.02174334]
39 : [0.00082]
40 : [0.2481435]
41 : [0.00547764]
42 : [0.00099727]
43 : [0.00793994]
44 : [0.0

In [38]:
# RandomForest_bidding Coherencia, Completitud, Congruencia
print("Estas son las metricas de coherencia, completitud y congruencia del randomForest de apuestas")
co = 0.0
cohe = []
# Red de apuestas (REVISAR PORQUE NO SE ELIMINAN LAS CARACTERISTICAS MENOS IMPORTANTES)
prediccion_original = randomForest_Bidding.predict(muestras_apuestas)
muestras_modificadas = np.copy(muestras_apuestas)
    # Eliminacio de caracteristicas no importantes para red de apuestas
muestras_modificadas[:, 0] = 0
muestras_modificadas[:, 6] = 0
muestras_modificadas[:, 4] = 0
prediccion_nueva = randomForest_Bidding.predict(muestras_modificadas)
print("La coherencia se enseña a continuacion por cada muestra: ")

for j in range(0, 256):
    
    co = coherencia_(prediccion_original[j], prediccion_nueva[j], objetivo_apuestas[j])
    cohe.append(co)
    print(j,":",co)

prediccion_original = randomForest_Bidding.predict(muestras_apuestas)
explicacion_error = []

for i in range(0, 256):
    exp = explicacion_del_modelo(10,randomForest_Bidding, muestras_apuestas[i], muestras_apuestas)
    exp2 = explicacion_del_modelo(10,randomForest_Bidding, muestras_modificadas[i], muestras_modificadas)
    explicacion_error.append(np.abs(exp - exp2))
explicacion_error = np.mean(explicacion_error)
comple = completitud_(prediccion_original,explicacion_error, objetivo_apuestas)
print("La completitud se muestra a continuacion:")
print(comple)

#Congruencia
congru = congruencia_(cohe)
print("La congruencia es: ", congru)

Estas son las metricas de coherencia, completitud y congruencia del randomForest de apuestas
La coherencia se enseña a continuacion por cada muestra: 
0 : 0
1 : 0
2 : 0
3 : 0
4 : 0
5 : 0
6 : 0
7 : 0
8 : 0
9 : 0
10 : 0
11 : 0
12 : 0
13 : 0
14 : 0
15 : 0
16 : 0
17 : 0
18 : 0
19 : 0
20 : 0
21 : 0
22 : 0
23 : 0
24 : 0
25 : 0
26 : 0
27 : 0
28 : 0
29 : 0
30 : 0
31 : 0
32 : 0
33 : 0
34 : 0
35 : 0
36 : 0
37 : 0
38 : 0
39 : 0
40 : 0
41 : 0
42 : 0
43 : 0
44 : 0
45 : 0
46 : 0
47 : 0
48 : 0
49 : 0
50 : 0
51 : 0
52 : 0
53 : 0
54 : 0
55 : 0
56 : 0
57 : 0
58 : 0
59 : 0
60 : 0
61 : 0
62 : 0
63 : 0
64 : 0
65 : 0
66 : 0
67 : 0
68 : 0
69 : 0
70 : 0
71 : 0
72 : 0
73 : 0
74 : 0
75 : 0
76 : 0
77 : 0
78 : 0
79 : 0
80 : 0
81 : 0
82 : 0
83 : 0
84 : 0
85 : 0
86 : 0
87 : 0
88 : 0
89 : 0
90 : 0
91 : 0
92 : 0
93 : 0
94 : 0
95 : 0
96 : 0
97 : 0
98 : 0
99 : 0
100 : 0
101 : 0
102 : 0
103 : 0
104 : 0
105 : 0
106 : 0
107 : 0
108 : 0
109 : 0
110 : 0
111 : 0
112 : 0
113 : 0
114 : 0
115 : 0
116 : 0
117 : 0
118 : 0
119 : 0

In [39]:
# Red_vinos Coherencia, Completitud, Congruencia
print("Estas son las metricas de coherencia, completitud y congruencia de la red neuronal de vinos")
co = 0.0
cohe = []
prediccion_original = []
prediccion_nueva = []
# Red de apuestas (REVISAR PORQUE NO SE ELIMINAN LAS CARACTERISTICAS MENOS IMPORTANTES)
prediccion_original_aux = red_vino.predict(muestras_vinos)
muestras_modificadas = np.copy(muestras_vinos)
    # Eliminacio de caracteristicas no importantes para red de apuestas
muestras_modificadas[:, 8] = 0
muestras_modificadas[:, 5] = 0
muestras_modificadas[:, 9] = 0
prediccion_nueva_aux = red_vino.predict(muestras_modificadas)
print("La coherencia se enseña a continuacion por cada muestra: ")
for j in range(0, 256):
    
    co = coherencia_(prediccion_original_aux[j], prediccion_nueva_aux[j], objetivo_vinos[j])
    cohe.append(co)
    print(j,":",co)

prediccion_original_aux = red_vino.predict(muestras_vinos)
explicacion_error = []
for i, j in zip(prediccion_original_aux,prediccion_nueva_aux):
    prediccion_nueva.append(obtener_indice_maximo(j))
    prediccion_original.append(obtener_indice_maximo(i))

for i in range(0, 256):
    exp = explicacion_del_modelo(10,red_vino, muestras_vinos[i], muestras_vinos)
    exp2 = explicacion_del_modelo(10,red_vino, muestras_modificadas[i], muestras_modificadas)
    explicacion_error.append(np.abs(exp - exp2))
explicacion_error = np.mean(explicacion_error)
comple = completitud_(prediccion_original,explicacion_error, objetivo_apuestas)
print("La completitud se muestra a continuacion:")
print(comple)
#Congruencia
congru = congruencia_(cohe)
print("La congruencia es: ", congru)

Estas son las metricas de coherencia, completitud y congruencia de la red neuronal de vinos
8/8 [==============================] - 0s 2ms/step
La coherencia se enseña a continuacion por cada muestra: 
0 : [0.20419872 0.48432875 0.22696644 0.0531636 ]
1 : [0.23944983 0.29443122 0.49190086 0.04198026]
2 : [0.18123555 0.22074386 0.36748017 0.03449922]
3 : [0.01846072 0.19277823 0.19397795 0.01726106]
4 : [0.28610411 0.17063317 0.388752   0.06798542]
5 : [0.22052538 0.45543408 0.21862753 0.01628117]
6 : [0.26791356 0.30660784 0.03619797 0.00249636]
7 : [0.05339545 0.02714816 0.01608087 0.01016651]
8 : [0.15412647 0.13015272 0.235981   0.04829825]
9 : [2.09748745e-04 2.09748348e-04 8.88346545e-19 3.05151468e-11]
10 : [0.23102847 0.26073727 0.4475286  0.04423719]
11 : [0.04653507 0.18629259 0.11092496 0.02883262]
12 : [0.17008477 0.27271402 0.40851665 0.03428215]
13 : [0.06866264 0.1502662  0.19275013 0.02617876]
14 : [0.39558184 0.44809525 0.0330795  0.01943382]
15 : [0.27844512 0.4068138  

8/8 [==============================] - 0s 2ms/step
La completitud se muestra a continuacion:
0.4765701036306565
La congruencia es:  0.0020257288165573205


In [40]:
# RandomForest_vinos Coherencia, Completitud, Congruencia
print("Estas son las metricas de coherencia, completitud y congruencia del randomForest de apuestas")
co = 0.0
cohe = []
# Red de apuestas (REVISAR PORQUE NO SE ELIMINAN LAS CARACTERISTICAS MENOS IMPORTANTES)
prediccion_original = randomForest_vinos.predict(muestras_vinos)
muestras_modificadas = np.copy(muestras_vinos)
    # Eliminacio de caracteristicas no importantes para red de apuestas
muestras_modificadas[:, 8] = 0
muestras_modificadas[:, 5] = 0
muestras_modificadas[:, 1] = 0
prediccion_nueva = randomForest_vinos.predict(muestras_modificadas)
print("La coherencia se enseña a continuacion por cada muestra: ")
for j in range(0, 256):
    
    co = coherencia_(prediccion_original[j], prediccion_nueva[j], objetivo_vinos[j])
    cohe.append(co)
    print(j,":",co)

prediccion_original = randomForest_vinos.predict(muestras_vinos)
explicacion_error = []

for i in range(0, 256):
    exp = explicacion_del_modelo(10,randomForest_vinos, muestras_vinos[i], muestras_vinos)
    exp2 = explicacion_del_modelo(10,randomForest_vinos, muestras_modificadas[i], muestras_modificadas)
    explicacion_error.append(np.abs(exp - exp2))
explicacion_error = np.mean(explicacion_error)
comple = completitud_(prediccion_original,explicacion_error, objetivo_apuestas)
print("La completitud se muestra a continuacion:")
print(comple)
#Congruencia
congru = congruencia_(cohe)
print("La congruencia es: ", congru)

Estas son las metricas de coherencia, completitud y congruencia del randomForest de apuestas
La coherencia se enseña a continuacion por cada muestra: 
0 : [0. 0. 0. 0.]
1 : [0. 0. 0. 0.]
2 : [1. 1. 1. 1.]
3 : [0. 0. 0. 0.]
4 : [1. 1. 1. 1.]
5 : [0. 0. 0. 0.]
6 : [1. 1. 1. 1.]
7 : [0. 0. 0. 0.]
8 : [0. 0. 0. 0.]
9 : [1. 1. 1. 1.]
10 : [2. 2. 2. 2.]
11 : [1. 1. 1. 1.]
12 : [1. 1. 1. 1.]
13 : [0. 0. 0. 0.]
14 : [0. 0. 0. 0.]
15 : [1. 1. 1. 1.]
16 : [1. 1. 1. 1.]
17 : [1. 1. 1. 1.]
18 : [0. 0. 0. 0.]
19 : [0. 0. 0. 0.]
20 : [1. 1. 1. 1.]
21 : [1. 1. 1. 1.]
22 : [1. 1. 1. 1.]
23 : [0. 0. 0. 0.]
24 : [1. 1. 1. 1.]
25 : [1. 1. 1. 1.]
26 : [0. 0. 0. 0.]
27 : [0. 0. 0. 0.]
28 : [0. 0. 0. 0.]
29 : [1. 1. 1. 1.]
30 : [1. 1. 1. 1.]
31 : [1. 1. 1. 1.]
32 : [1. 1. 1. 1.]
33 : [0. 0. 0. 0.]
34 : [1. 1. 1. 1.]
35 : [1. 1. 1. 1.]
36 : [0. 0. 0. 0.]
37 : [1. 1. 1. 1.]
38 : [1. 1. 1. 1.]
39 : [1. 1. 1. 1.]
40 : [1. 1. 1. 1.]
41 : [1. 1. 1. 1.]
42 : [1. 1. 1. 1.]
43 : [1. 1. 1. 1.]
44 : [1. 1. 1. 1.]
45 :

La coherencia sirve para ver en cuanto varia el modelo tomando muestras perturbadas y sin perturbar, esto nos da una idea de cuanto de bueno es nuestro modelo y la completitud nos muestra un porcentaje de fallo con respecto a los fallos de prediccion, mientras que la congruencia nos calcula la desviacion tipica de estos fallos.

# Conclusión y Resultados:

Como podemos observar en los resultados obtenidos, podemos decir que las metricas se han superado de manera exitosa pudiendo hacer un estudio sobre las explicaciones obtenidas con respecto a las muestras perturbadas de cada modelo y tambien sacar posibles estrategias para la mejora de los modelos mediante el uso de nuestro algoritmo XAI LIME, gracias a todos estos calculos la forma de interpretar modelos de caja negra es mucho mas llevadera que si tuvieramos que realizar un estudio a mano.

Con el uso de estas metricas cuyos resultados estan disponibles encima de este apartado, observamos que los resultados son coherentes, tambien nos guian a la hora de saber como se comporta el modelo cuando una muestra esta desequilibrada, nos confirma que a iguales muestras el modelo no se comporta de forma diferente y obtenemos informacion relevante sobre los datos de estos modelos como las desviaciones tipicas de sus resultados

Como conclusion de este proyecto podemos afirmar que el Algoritmo LIME es de gran utilidad para poder sacar conclusiones aproximadas sobre el comportamiento de modelos cuya funcionalidad se desconoce y con esto confirmamos que las XAI son necesarias para poder conocer el comportamiento de estos tipos de algoritmos.